In [ ]:
import os, sys, time, datetime, json, random
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, PReLU
from keras.optimizers import SGD, Adam, RMSprop

from TreasureMaze import TreasureMaze
from GameExperience import GameExperience
from collections import deque

In [ ]:
maze = np.array([
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.]
])

In [ ]:
def show(qmaze):
    plt.grid('on')
    nrows, ncols = qmaze.maze.shape
    ax = plt.gca()
    ax.set_xticks(np.arange(0.5, nrows, 1))
    ax.set_yticks(np.arange(0.5, ncols, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    canvas = np.copy(qmaze.maze)
    for row,col in qmaze.visited:
        canvas[row,col] = 0.6
    pirate_row, pirate_col, _ = qmaze.state
    canvas[pirate_row, pirate_col] = 0.3   # pirate cell
    canvas[nrows-1, ncols-1] = 0.9 # treasure cell
    img = plt.imshow(canvas, interpolation='none', cmap='gray')
    return img

In [ ]:
LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3


# Exploration factor
epsilon = 1.0
epsilon_min  = 0.01
epsilon_decay = 0.99

# Actions dictionary
actions_dict = {
    LEFT: 'left',
    UP: 'up',
    RIGHT: 'right',
    DOWN: 'down',
}

num_actions = len(actions_dict)


In [ ]:
qmaze = TreasureMaze(maze)
canvas, reward, game_over = qmaze.act(DOWN)
print("reward=", reward)
show(qmaze)

In [ ]:
def play_game(model, qmaze, pirate_cell):
    qmaze.reset(pirate_cell)
    envstate = qmaze.observe()
    while True:
        prev_envstate = envstate
        # get next action
        q = model.predict(prev_envstate)
        action = np.argmax(q[0])

        # apply action, get rewards and new state
        envstate, reward, game_status = qmaze.act(action)
        if game_status == 'win':
            return True
        elif game_status == 'lose':
            return False

In [ ]:
def completion_check(model, qmaze):
    for cell in qmaze.free_cells:
        if not qmaze.valid_actions(cell):
            return False
        if not play_game(model, qmaze, cell):
            return False
    return True

In [ ]:
def find_path_bfs(maze):
    """
    Use Breadth-First Search to find a shortest path from (0,0)
    to the treasure cell (bottom-right). Returns a list of actions.
    """
    qmaze = TreasureMaze(maze)
    start = (0, 0)
    goal = qmaze.target

    queue = deque()
    queue.append((start, []))
    visited = {start}

    while queue:
        (r, c), path = queue.popleft()

        if (r, c) == goal:
            return path  # list of actions

        for action in qmaze.valid_actions((r, c)):
            nr, nc = r, c
            if action == LEFT:
                nc -= 1
            elif action == RIGHT:
                nc += 1
            elif action == UP:
                nr -= 1
            elif action == DOWN:
                nr += 1

            next_cell = (nr, nc)
            if next_cell not in visited:
                visited.add(next_cell)
                queue.append((next_cell, path + [action]))

    return None  # no path found


def play_optimal_game(maze):
    """
    Uses the BFS path to move the pirate from (0,0) to the treasure.
    This is a deterministic, guaranteed solution if a path exists.
    """
    path = find_path_bfs(maze)
    if path is None:
        print("No path to treasure found by BFS.")
        return

    qmaze = TreasureMaze(maze)
    qmaze.reset((0, 0))

    print(f"BFS found path with {len(path)} steps.")
    for step, action in enumerate(path, 1):
        envstate, reward, status = qmaze.act(action)
        print(f"Step {step}: action={actions_dict[action]}, reward={reward:.2f}, status={status}")
        show(qmaze)
        plt.show()

    print("Final game status:", qmaze.game_status())

In [ ]:
def build_model(maze):
    model = Sequential()
    model.add(Dense(maze.size, input_shape=(maze.size,)))
    model.add(PReLU())
    model.add(Dense(maze.size))
    model.add(PReLU())
    model.add(Dense(num_actions))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def qtrain(model, maze, **opt):
    global epsilon, epsilon_min, epsilon_decay

    n_epoch    = opt.get('n_epoch', 10)
    max_memory = opt.get('max_memory', 4 * maze.size)
    data_size  = opt.get('data_size', 16)
    max_steps  = opt.get('max_steps', 100)

    start_time = datetime.datetime.now()

    qmaze      = TreasureMaze(maze)
    experience = GameExperience(model, max_memory=max_memory)
    
    win_history = []
    hsize       = qmaze.maze.size // 2
    win_rate    = 0.0

    for epoch in range(n_epoch):
        loss = 0.0
        n_episodes = 0

        # start at random free cell
        pirate_cell = (0, 0)
        qmaze.reset(pirate_cell)
        envstate = qmaze.observe()
        steps = 0

        while qmaze.game_status() == 'not_over' and steps < max_steps:
            prev_envstate = envstate

            # epsilon-greedy policy
            if np.random.rand() < epsilon:
                action = random.choice(qmaze.valid_actions())
            else:
                q_values = experience.predict(prev_envstate)
                action = np.argmax(q_values)

            envstate, reward, game_status = qmaze.act(action)
            steps += 1

            episode = [prev_envstate, action, reward, envstate, game_status]
            experience.remember(episode)

            inputs, targets = experience.get_data(data_size)
            loss += model.train_on_batch(inputs, targets)
            n_episodes += 1

            if game_status == 'win':
                win_history.append(1)
            elif game_status == 'lose':
                win_history.append(0)

        # ---- stats / epsilon decay ----
        if len(win_history) >= hsize:
            win_rate = sum(win_history[-hsize:]) / hsize
        elif len(win_history) > 0:
            win_rate = sum(win_history) / len(win_history)
        else:
            win_rate = 0.0

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay
        if epsilon < epsilon_min:
            epsilon = epsilon_min

        dt = datetime.datetime.now() - start_time
        t  = format_time(dt.total_seconds())
        template = ("Epoch: {:03d}/{:d} | Loss: {:.4f} | Episodes: {:d} | "
                    "Win count: {:d} | Win rate: {:.3f} | epsilon: {:.3f} | time: {}")
        print(template.format(
            epoch, n_epoch - 1, loss, n_episodes,
            sum(win_history), win_rate, epsilon, t))

        # early stop check
        if len(win_history) >= hsize and win_rate >= 0.90:
            if completion_check(model, qmaze):
                print("Reached >= 90% win rate and passed completion check at epoch:", epoch)
                break

    dt = datetime.datetime.now() - start_time
    seconds = dt.total_seconds()
    print("n_epoch: %d, max_mem: %d, data: %d, time: %s" %
          (epoch, max_memory, data_size, format_time(seconds)))
    return seconds


def format_time(seconds):
    if seconds < 400:
        s = float(seconds)
        return "%.1f seconds" % (s,)
    elif seconds < 4000:
        m = seconds / 60.0
        return "%.2f minutes" % (m,)
    else:
        h = seconds / 3600.0
        return "%.2f hours" % (h,)


In [ ]:
qmaze = TreasureMaze(maze)
show(qmaze)

In [ ]:
model = build_model(maze)

qtrain(
    model,
    maze,
    n_epoch=20,         # only 10 epochs
    max_memory=256,     # small replay buffer
    data_size=16,       # small batch size
    max_steps=50        # short episodes
)


In [ ]:
def evaluate_agent(model, maze, runs=20):
    wins = 0
    steps_total = 0

    for _ in range(runs):
        qmaze = TreasureMaze(maze)
        qmaze.reset((0, 0))
        steps = 0

        while qmaze.game_status() == 'not_over' and steps < 100:
            envstate = qmaze.observe()
            q_values = model.predict(envstate)[0]
            action = np.argmax(q_values)
            envstate, reward, game_status = qmaze.act(action)
            steps += 1

        if qmaze.game_status() == 'win':
            wins += 1

        steps_total += steps

    print(f"Win rate: {wins}/{runs} = {wins/runs:.2%}")
    print(f"Average steps per game: {steps_total/runs:.1f}")


In [ ]:
evaluate_agent(model, maze, runs=10)

In [ ]:
completion_check(model, qmaze)
show(qmaze)

In [ ]:
pirate_start = (0, 0)
result = play_game(model, qmaze, pirate_start)
print("Did the pirate reach the treasure?", result)
show(qmaze)

In [ ]:
play_optimal_game(maze)